In [ ]:
from xgboost import XGBRegressor
from config import TRAIN_END_DATE, VALID_END_DATE
import pandas as pd
import numpy as np
from sklearn.model_selection import RandomizedSearchCV
import yfinance as yf

In [29]:
# Some Helper functions
def rmse(y_true, y_pred):
    """
    Root Mean Squared Error: measures magnitude error sensitive to outliers 
    rmse computes the square root of the mean of the squared differences between 
    predicted and actual values
    """
    return np.sqrt(np.mean((y_true - y_pred) ** 2))

def spearman_rank_corr(y_true, y_pred):
    """
    Spearman Rank Correlation: measures ranking accuracy
    """
    # if no samples, return NaN
    if len(y_true) == 0:
        return np.nan
    return pd.Series(y_true).corr(pd.Series(y_pred), method="spearman")

In [30]:
# Importing datasets from model_data_builder
df = pd.read_csv("model_data.csv")
# print(df.columns)
df["period_end"] = pd.to_datetime(df["period_end"])
target_col = "next_excess_ret"
df.head()

,SP Identifier,PERMNO,period_end,Monthly Price,Monthly Total Return,Monthly Total Return Excluding Dividends,Shares Outstanding (CRSP),avail_from,avail_to,Fiscal Year,...,HML,Mkt_RF_ann,SMB_ann,HML_ann,next_ret_12m,next_excess_ret,excess_ret,beta_mkt_rf,beta_smb,beta_hml
0,1004,54594.0,2001-05-31,14.00,0.246661,0.246661,26932.0,2001-04-30,2002-04-29,2000,...,0.131116,-0.064919,0.035576,0.131116,-0.182857,-0.207470,NaN,NaN,NaN,NaN
1,1004,54594.0,2002-05-31,11.44,-0.100211,-0.102041,26860.0,2002-04-30,2003-04-29,2001,...,0.031405,-0.078152,-0.003087,0.031405,-0.606643,-0.620868,-0.205861,NaN,NaN,NaN
2,1004,54594.0,2003-05-31,4.50,0.159794,0.159794,31850.0,2003-04-30,2004-04-29,2002,...,0.051667,0.063062,0.024852,0.051667,1.128889,1.119499,-0.620311,NaN,NaN,NaN
3,1004,54594.0,2004-05-31,9.58,-0.056158,-0.056158,32245.0,2004-04-30,2005-04-29,2003,...,-0.000590,-0.009844,0.030379,-0.000590,0.674322,0.654851,1.119541,NaN,NaN,NaN
4,1004,54594.0,2005-05-31,16.04,0.088934,0.088934,32438.0,2005-04-30,2006-04-29,2004,...,-0.013612,0.035983,-0.032309,-0.013612,0.501247,0.462968,0.653306,-1.400609,13.90712,-29.869485


In [31]:
# Columns that contain future information at t+1, excluding them from X
biasedcols = [col for col in df.columns if col.startswith("next_")]
id_cols = ["SP Identifier", "PERMNO"]
filtered_cols = biasedcols + id_cols + ["period_end"]
features = [col for col in df.columns if col not in filtered_cols]
# df = df.dropna(subset = [target_col])
df.head()

,SP Identifier,PERMNO,period_end,Monthly Price,Monthly Total Return,Monthly Total Return Excluding Dividends,Shares Outstanding (CRSP),avail_from,avail_to,Fiscal Year,...,HML,Mkt_RF_ann,SMB_ann,HML_ann,next_ret_12m,next_excess_ret,excess_ret,beta_mkt_rf,beta_smb,beta_hml
0,1004,54594.0,2001-05-31,14.00,0.246661,0.246661,26932.0,2001-04-30,2002-04-29,2000,...,0.131116,-0.064919,0.035576,0.131116,-0.182857,-0.207470,NaN,NaN,NaN,NaN
1,1004,54594.0,2002-05-31,11.44,-0.100211,-0.102041,26860.0,2002-04-30,2003-04-29,2001,...,0.031405,-0.078152,-0.003087,0.031405,-0.606643,-0.620868,-0.205861,NaN,NaN,NaN
2,1004,54594.0,2003-05-31,4.50,0.159794,0.159794,31850.0,2003-04-30,2004-04-29,2002,...,0.051667,0.063062,0.024852,0.051667,1.128889,1.119499,-0.620311,NaN,NaN,NaN
3,1004,54594.0,2004-05-31,9.58,-0.056158,-0.056158,32245.0,2004-04-30,2005-04-29,2003,...,-0.000590,-0.009844,0.030379,-0.000590,0.674322,0.654851,1.119541,NaN,NaN,NaN
4,1004,54594.0,2005-05-31,16.04,0.088934,0.088934,32438.0,2005-04-30,2006-04-29,2004,...,-0.013612,0.035983,-0.032309,-0.013612,0.501247,0.462968,0.653306,-1.400609,13.90712,-29.869485


In [32]:
# keep numeric features only to avoid object dtype issues
X_full = df[features]
# keep only the numeric columns
numeric_cols = X_full.select_dtypes(include=[np.number]).columns.tolist()
# Fill the missing values with 0.0
X = X_full[numeric_cols].fillna(0.0)
y = df[target_col].values

train_mask = df["period_end"] <= pd.to_datetime(TRAIN_END_DATE)
valid_mask = (df["period_end"] > pd.to_datetime(TRAIN_END_DATE)) & (df["period_end"] <= pd.to_datetime(VALID_END_DATE))
test_mask = df["period_end"] > pd.to_datetime(VALID_END_DATE)

In [207]:
# Set up some fixed parameters
XGBoost_model = XGBRegressor(
    n_estimators = 1500,
    random_state = 44,
    eval_metric="rmse",
    early_stopping_rounds = 150,
    n_jobs = -1
)
# Use GridSearchCV to tune the remaining parameters
param_grid = {
    # Controls the complexity of the model
    "max_depth": [2, 3,4, 5, 6],
    # Control the learning rate (speed)
    "learning_rate": [0.005, 0.01, 0.02, 0.03, 0.04, 0.05],
    # Controls the percentage of firms (rows) each individual tree is allowed to see
    "subsample": [0.6, 0.7, 0.8, 0.9],
    # Controls the percentage of features(columns) each individual tree is allowed to use
    "colsample_bytree": [0.6, 0.7, 0.8, 0.9],
    # Regularization parameters - prevent overfitting
    "min_child_weight": [5, 10, 20],
    # L1 Regularization (Lasso) 
    "reg_alpha": [0, 0.1, 1, 10],
    # L2 Regularization (Ridge)   
    "reg_lambda": [0, 1, 10],
}

# Model Trainging
model = RandomizedSearchCV(
    estimator = XGBoost_model,
    param_distributions = param_grid,
    n_iter = 100,
    scoring = "neg_mean_squared_error",
    n_jobs = -1,
    verbose = 1,
    random_state = 44
)

In [208]:
# Train the XGBoost model before running validation metrics
model.fit(
    X[train_mask],
    y[train_mask],
    eval_set=[(X[valid_mask], y[valid_mask])],
    verbose = False,
)
# Extract the tuned model
print(f"Best Parameters Found: {model.best_params_}")
best_model = model.best_estimator_

Fitting 5 folds for each of 100 candidates, totalling 500 fits
Best Parameters Found: {'subsample': 0.7, 'reg_lambda': 1, 'reg_alpha': 1, 'min_child_weight': 10, 'max_depth': 4, 'learning_rate': 0.05, 'colsample_bytree': 0.9}


In [209]:
# Configuring the Final Model
x_train_validation = pd.concat([X[train_mask], X[valid_mask]])
y_train_validation = np.concatenate([y[train_mask], y[valid_mask]])
# Train the final model on the entire training set
Final_Model = XGBRegressor(
    **model.best_params_,
    n_estimators = 1500,
    n_jobs = -1,
    random_state = 66
)
print("Training Final Production Model...")
Final_Model.fit(x_train_validation, y_train_validation, verbose=False)
print("Training Complete.")


Training Final Production Model...
Training Complete.


In [210]:
y_test = y[test_mask]
# Writes the results on the testing data into the dataframe
df.loc[test_mask, "predicted_next_excess_ret"] = Final_Model.predict(X[test_mask])
# Creates the results dataframe
predicted_df = df.loc[test_mask, ["PERMNO", "period_end", target_col, "predicted_next_excess_ret"]].copy()
# drop the missing values
predicted_df = predicted_df.dropna(subset=["predicted_next_excess_ret"])

# Rank stocks by Score 
predicted_df["rank"] = predicted_df.groupby("period_end")["predicted_next_excess_ret"].rank(method="first", ascending=False)

# Select the top 5 stocks to form the portfolio
Portfolio = predicted_df[predicted_df["rank"] <= 5].copy()
Portfolio = Portfolio.rename(columns = {target_col: "actual_next_excess_ret"})

Portfolio.to_csv("top5.csv", index=False)

In [ ]:
portfolio_path = "top5.csv"   # or "top10_long_only.csv"
crsp_path = "new_data.csv"
portfolio = pd.read_csv(portfolio_path)
portfolio["period_end"] = pd.to_datetime(portfolio["period_end"], errors="coerce")
portfolio = portfolio.dropna(subset=["period_end", "PERMNO"]).copy()
crsp = pd.read_csv(crsp_path)
crsp["MthCalDt"] = pd.to_datetime(crsp["MthCalDt"], errors="coerce")
crsp["period_end"] = crsp["MthCalDt"].dt.to_period("M").dt.to_timestamp("M")
firm_lst = portfolio["PERMNO"].unique()
firm_lst[1:6]
data = crsp[crsp["PERMNO"].isin(firm_lst[1:6])]
names = data["conm"].unique()
names
# Analog Devices Inc (ADI)
# ApoGee Enterprises Inc (APOG)
#Applied Materials, Inc. (AMAT)
# AutoDesk Inc (ADSK)
# ENERPAC TOOL GROUP CORP (EPAC)

array(['ENERPAC TOOL GROUP CORP', 'APOGEE ENTERPRISES INC',
       'APPLIED MATERIALS INC', 'ANALOG DEVICES INC', 'AUTODESK INC'],
      dtype=object)

In [24]:
data = data[["conm", "MthCalDt", "MthRet"]]
data = data.pivot_table(index="MthCalDt", columns="conm", values="MthRet")
# data.head()
data.to_csv("selected_data.csv")

In [62]:
tickers = ["ADI", "APOG", "AMAT", "ADSK", "EPAC"]
prices = yf.download(
    tickers=tickers,
    start="2020-01-01",
    end="2025-01-01",
    group_by="ticker",
    auto_adjust=True,
)
prices.to_csv("historical_prices.csv")
prices.head()

[*********************100%***********************]  5 of 5 completed


Ticker           AMAT                                                 EPAC  \
Price            Open       High        Low      Close   Volume       Open   
Date                                                                         
2020-01-02  58.583679  59.197864  58.300210  58.772659  6647900  25.947665   
2020-01-03  57.345876  58.328571  57.223041  57.837223  5415200  25.472105   
2020-01-06  56.552158  57.128546  56.032464  56.589954  8586000  25.402752   
2020-01-07  56.949025  58.678187  56.826187  58.224636  7970400  25.244232   
2020-01-08  58.300219  58.772668  57.875014  58.186832  4364400  25.046084   

Ticker                                               ...       APOG  \
Price            High        Low      Close  Volume  ...       Open   
Date                                                 ...              
2020-01-02  26.096277  25.511736  25.917942  407900  ...  28.489023   
2020-01-03  25.699978  25.373031  25.699978  353400  ...  28.331818   
2020-01-06  25.501827  25.105529  25.422567  328600  ...  28.366754   
2020-01-07  25.244232  24.917285  25.006453  573800  ...  28.480291   
2020-01-08  25.204604  24.877657  25.075806  258900  ...  28.419152   

Ticker                                                     ADSK              \
Price            High        Low      Close  Volume        Open        High   
Date                                                                          
2020-01-02  28.611293  28.253215  28.541426  219800  184.210007  187.889999   
2020-01-03  28.768499  28.104746  28.681164  235200  184.490005  186.410004   
2020-01-06  28.707364  27.930073  28.628761  329900  183.779999  187.250000   
2020-01-07  28.943171  28.401688  28.506491  189800  186.130005  188.169998   
2020-01-08  29.135307  28.419152  28.908232  263300  188.220001  190.500000   

Ticker                                       
Price              Low       Close   Volume  
Date                                         
2020-01-02  181.880005  187.830002  1515000  
2020-01-03  183.779999  184.949997   737900  
2020-01-06  183.160004  187.119995  1021900  
2020-01-07  185.100006  187.500000   897200  
2020-01-08  187.029999  189.949997  1314800  

[5 rows x 25 columns]

In [63]:
# right now , we have a multi-index dataframe
# we need to pivot it to a long formate
level_0 = prices.columns.get_level_values(0).unique()
close_df = pd.DataFrame(index=prices.index)
print(close_df.head())

for ticker in level_0:
    close_df[ticker] = prices[ticker]["Close"]

print(close_df) # creating a dataframe with the closing prices


Empty DataFrame
Columns: []
Index: [2020-01-02 00:00:00, 2020-01-03 00:00:00, 2020-01-06 00:00:00, 2020-01-07 00:00:00, 2020-01-08 00:00:00]
                  AMAT       EPAC         ADI       APOG        ADSK
Date                                                                
2020-01-02   58.772659  25.917942  108.129166  28.541426  187.830002
2020-01-03   57.837223  25.699978  106.225693  28.681164  184.949997
2020-01-06   56.589954  25.422567  104.977692  28.628761  187.119995
2020-01-07   58.224636  25.006453  107.366005  28.506491  187.500000
2020-01-08   58.186832  25.075806  108.335693  28.908232  189.949997
...                ...        ...         ...        ...         ...
2024-12-24  166.701294  42.348183  214.551743  70.354195  301.230011
2024-12-26  165.889420  42.128399  214.522232  71.408485  300.279999
2024-12-27  165.176559  41.339180  213.469604  70.617767  297.589996
2024-12-30  162.018173  41.349171  209.111465  69.573242  297.529999
2024-12-31  161.018188  41.0494